In [ ]:
import sympy as sp
import numpy as np
import matplotlib.pyplot as plt

### 1. Formulate the statement of the interpolation problem with Cubic Spline [mathematical formula]

In [ ]:
### YOUR TEXT/CODE HERE










### 2. Formulate the functional and differential compatibility conditions [mathematical formula]

In [ ]:
### YOUR TEXT/CODE HERE










### 3. Formulate stitching conditions [mathematical formula]

In [ ]:
### YOUR TEXT/CODE HERE










### 4. Justify why these conditions provide you with the required smoothness [thesis text, no more than 500 characters]

In [ ]:
### YOUR TEXT/CODE HERE










### 5. Derive dependency formula: the dependence of the second derivatives at the grid nodes on the increment of the function (the function values difference on the grid nodes). [Mathematical formulas derivation. Detailed, with clear transitions]

In [ ]:
### YOUR TEXT/CODE HERE










### 6. Create a system of equations using this formula [Matrix representation. Mathematical formulas]

In [ ]:
### YOUR TEXT/CODE HERE










### 7. Explain what is an unknown variable in this system. whether the system is closed with respect to an unknown variable. What is missing for closure. [Text, no more than 200 characters]

In [ ]:
### YOUR TEXT/CODE HERE










### 8. Bring this matrix to the appropriate form to use the Tridiagonal matrix algorithm [Mathematical derivation. Use Gauss Elimination]

In [ ]:
### YOUR TEXT/CODE HERE










### 9. Derive formulas of direct pass and reverse pass of Tridiagonal matrix algorithm [Mathematical formals]

In [ ]:
### YOUR TEXT/CODE HERE










### 10. Implement code prototype of the future algorithm implementation. Classes/methods (if you use OOP), functions. The final implementation (on language chosen by you) should not differ from the functions declared in the prototype. [Python code]

In [ ]:
### YOUR CODE HERE










### 11. Derive formula of Cubic Spline method error [Mathematical formulas]

In [ ]:
### YOUR TEXT/CODE HERE










### 12. Rate the complexity of the algorithm [Text, and rate in terms of big O, no more than 100 characters]

In [ ]:
### YOUR TEXT/CODE HERE










### Congrats!